---
title: "A look at different scenarios - Gains and Losses"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx: 
    fig-height: 11.6 in
---

In [1]:
import pandas as pd
from fagrants_module import fagrants_gccsa as fagrants

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].mean()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / grouped_df[f"ERP_{year}"].sum()

        return_dict[f"Grant Delta_{year}"] = weighted_mean

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def grants_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "LGA"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    pop_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Population_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Population",
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted_per_capita = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita ($)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    sim_grants_melted["Expected Change in GPG Per Capita ($)"] = sim_grants_melted_per_capita["Expected Change in GPG Per Capita ($)"]

    sim_grants_melted["Population"] = pop_melted["Population"]

    return sim_grants_melted

In [2]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.20, 0.30)

sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.10, 0.30)

sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.00, 0.30)

In [3]:
sim_grants_30_melted = grants_delta(sim_grants_30)

sim_grants_20_melted = grants_delta(sim_grants_20)

sim_grants_10_melted = grants_delta(sim_grants_10)

sim_grants_0_melted = grants_delta(sim_grants_0)

In [4]:
from datetime import datetime as tm

with pd.ExcelWriter(f"./Data/Output/Simulation Results {tm.today().strftime("%d_%m_%Y")}.xlsx") as writer:
    sim_grants_30_melted.to_excel(writer, sheet_name = "30% Minimum Grant")
    sim_grants_20_melted.to_excel(writer, sheet_name = "20% Minimum Grant")
    sim_grants_10_melted.to_excel(writer, sheet_name = "10% Minimum Grant")
    sim_grants_0_melted.to_excel(writer, sheet_name = "0% Minimum Grant")

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 1)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

def display_data(data, state):
    sub = data[data["Jurisdiction"] == state]

    k = sub.groupby(["Jurisdiction", "ACLG", "Year"]).agg(
        {
            "Expected Change in GPG ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Expected Change in GPG Per Capita ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Population": 'sum'
        }
    )

    display(k)

## A 20% Per Capita Minimum Grant

### New South Wales

In [6]:
display_data(sim_grants_20_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                    558839.5   
                     2026                    580321.8   
                     2027                    601572.1   
                     2028                    623151.5   
                RAM  2025                    524289.7   
                     2026                    544799.7   
                     2027                    565064.6   
                     2028                    585616.7   
                RAS  2025                    726324.0   
                     2026                    752173.7   
                     2027                    781749.4   
                     2028                    810088.6   
                RAV  2025                    564304.4   
                     2026                    585428.5   
                     2027                    608617.9   
                     2028                    630929.8   
                RTL  2025                    828164.1   
                     2026                    858960.9   
                     2027                    887476.9   
                     2028                    922655.2   
                RTM  2025                    921292.7   
                     2026                    950500.5   
                     2027                    987852.3   
                     2028                   1031463.7   
                RTX  2025                     38523.5   
                     2026                     39965.3   
                     2027                     41375.6   
                     2028                     42996.4   
                UCC  2025                  -2082509.3   
                     2026                  -2164906.0   
                     2027                  -2248760.1   
                     2028                  -2334804.7   
                UDL  2025                   -711911.5   
                     2026                   -740081.1   
                     2027                   -768746.2   
                     2028                   -798161.6   
                UDM  2025                   -393809.0   
                     2026                   -409439.4   
                     2027                   -425309.1   
                     2028                   -441619.0   
                UDS  2025                   -114141.4   
                     2026                   -118748.1   
                     2027                   -123413.9   
                     2028                   -128146.1   
                UDV  2025                  -2218240.9   
                     2026                  -2306393.9   
                     2027                  -2396312.3   
                     2028                  -2488009.4   
                UFL  2025                   -638714.8   
                     2026                   -664739.1   
                     2027                   -690320.2   
                     2028                   -717821.3   
                UFM  2025                   -372636.7   
                     2026                   -388157.6   
                     2027                   -404232.4   
                     2028                   -420005.0   
                UFV  2025                  -1201471.8   
                     2026                  -1252872.6   
                     2027                  -1304130.0   
                     2028                  -1357894.3   
                URL  2025                    151228.8   
                     2026                    158557.3   
                     2027                    167755.2   
                     2028                    177643.9   
                URM  2025                    273608.4   
                     2026                    286127.9   
                     2027                    297836.6   
                     2028                    310461.4   
                URS  2025            

### Victoria

In [7]:
display_data(sim_grants_20_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    424492.2   
                  2026                    442319.2   
                  2027                    461017.8   
                  2028                    480081.7   
             RAM  2025                    473788.8   
                  2026                    492849.5   
                  2027                    513640.2   
                  2028                    534970.5   
             RAV  2025                    407456.8   
                  2026                    425808.7   
                  2027                    444558.3   
                  2028                    464080.7   
             UCC  2025                  -1622527.7   
                  2026                  -1693812.6   
                  2027                  -1765786.3   
                  2028                  -1839477.0   
             UDL  2025                   -838333.4   
                  2026                   -875212.1   
                  2027                   -912774.2   
                  2028                   -950749.4   
             UDV  2025                  -1005794.3   
                  2026                  -1052022.3   
                  2027                  -1099803.9   
                  2028                  -1145979.3   
             UFL  2025                    288754.4   
                  2026                    291199.8   
                  2027                    295878.8   
                  2028                    304354.2   
             UFM  2025                     -6545.5   
                  2026                     -7619.3   
                  2027                     -9124.2   
                  2028                     -7872.6   
             UFS  2025                    310776.8   
                  2026                    324352.1   
                  2027                    338709.0   
                  2028                    350510.6   
             UFV  2025                   -133870.1   
                  2026                   -143229.7   
                  2027                   -153072.1   
                  2028                   -166450.9   
             URL  2025                    837821.1   
                  2026                    877590.1   
                  2027                    920356.4   
                  2028                    962072.0   
             URM  2025                    685880.7   
                  2026                    716914.1   
                  2027                    747384.9   
                  2028                    778853.0   
             URS  2025                    574892.0   
                  2026                    600852.8   
                  2027                    629336.6   
                  2028                    654656.0   
             URV  2025                    305145.0   
                  2026                    337138.6   
                  2027                    360863.6   
                  2028                    383581.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                   60.4     50462.0  
                  2026                                   62.5     50851.0  
                  2027                                   64.6     51217.0  
                  2028                                   66.8     51602.0  
             RAM  2025                                  121.0      3917.0  
                  2026                                  124.9      3947.0  
                  2027                                  129.2      3976.0  
                  2028                                  133.6      4005.0  
             RAV  2025                                   26.4    246987.0  
                  2026                                 

### Queensland

In [8]:
display_data(sim_grants_20_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   1356134.8   
                  2026                   1404182.4   
                  2027                   1463763.3   
                  2028                   1519110.3   
             RAV  2025                    833876.1   
                  2026                    870121.4   
                  2027                    903062.7   
                  2028                    939058.3   
             RTL  2025                   1160340.1   
                  2026                   1207226.7   
                  2027                   1253568.1   
                  2028                   1302204.0   
             RTM  2025                    852297.1   
                  2026                    886897.6   
                  2027                    922030.5   
                  2028                    957231.9   
             RTS  2025                    841916.8   
                  2026                    875996.4   
                  2027                    911450.9   
                  2028                    946434.7   
             RTX  2025                    837035.6   
                  2026                    871305.6   
                  2027                    905619.6   
                  2028                    939995.4   
             UCC  2025                 -12019933.1   
                  2026                 -12536627.4   
                  2027                 -13047239.9   
                  2028                 -13569107.2   
             UFM  2025                   -353126.0   
                  2026                   -367805.1   
                  2027                   -382330.9   
                  2028                   -397264.8   
             UFS  2025                   -161491.8   
                  2026                   -168547.9   
                  2027                   -175643.4   
                  2028                   -182744.3   
             URL  2025                   -988898.6   
                  2026                  -1027295.6   
                  2027                  -1065786.1   
                  2028                  -1104964.1   
             URM  2025                   -260667.4   
                  2026                   -270202.1   
                  2027                   -279932.2   
                  2028                   -289465.2   
             URS  2025                    255610.6   
                  2026                    267424.8   
                  2027                    278332.8   
                  2028                    288100.5   
             URV  2025                  -3649633.3   
                  2026                  -3797448.6   
                  2027                  -3944761.0   
                  2028                  -4094976.7   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  301.9      4492.0  
                  2026                                  308.7      4548.0  
                  2027                                  318.1      4601.0  
                  2028                                  326.4      4654.0  
             RAV  2025                                   62.6    105395.0  
                  2026                                   64.5    106725.0  
                  2027                                   66.2    107956.0  
                  2028                                   68.0    109216.0  
             RTL  2025                                  219.7     47507.0  
                  2026                                  225.7     48105.0  
                  2027                                  231.6     48660.0  
                  2028                                  237.6     49227.0  
             RTM  2025                     

### South Australia

In [9]:
display_data(sim_grants_20_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    212365.0   
                     2026                    219396.4   
                     2027                    226739.6   
                     2028                    234562.6   
                RAM  2025                    162894.1   
                     2026                    168270.6   
                     2027                    173834.4   
                     2028                    179011.5   
                RAS  2025                    134364.6   
                     2026                    138713.8   
                     2027                    143420.0   
                     2028                    148067.9   
                RAV  2025                    291646.7   
                     2026                    302093.7   
                     2027                    312503.2   
                     2028                    324415.8   
                RSG  2025                    -38900.3   
                     2026                    -39847.5   
                     2027                    -40783.9   
                     2028                    -41777.7   
                UCC  2025                   -250849.5   
                     2026                   -259447.9   
                     2027                   -268100.8   
                     2028                   -277072.5   
                UDL  2025                   -974991.1   
                     2026                  -1008276.2   
                     2027                  -1041976.6   
                     2028                  -1076810.5   
                UDM  2025                   -450446.0   
                     2026                   -465910.0   
                     2027                   -481442.5   
                     2028                   -497482.4   
                UDS  2025                   -172712.9   
                     2026                   -178634.9   
                     2027                   -184595.9   
                     2028                   -190770.8   
                UDV  2025                   -800878.7   
                     2026                   -829321.2   
                     2027                   -859995.8   
                     2028                   -890304.4   
                UFL  2025                    957825.3   
                     2026                    990201.9   
                     2027                   1012680.9   
                     2028                   1042917.5   
                UFM  2025                   -376187.4   
                     2026                   -389078.5   
                     2027                   -402069.2   
                     2028                   -415515.3   
                UFS  2025                     23487.6   
                     2026                     25039.2   
                     2027                     27676.5   
                     2028                     29237.1   
                UFV  2025                   -909544.5   
                     2026                   -939387.5   
                     2027                   -971142.4   
                     2028                  -1009807.1   
                URM  2025                   -236594.1   
                     2026                   -245491.0   
                     2027                   -253250.5   
                     2028                   -262378.4   
                URS  2025                    273826.0   
                     2026                    283808.0   
                     2027                    294825.0   
                     2028                    305642.3   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   25.0     83192.0  
                     2026  

### Western Australia

In [10]:
display_data(sim_grants_20_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    200070.5   
                       2026                    209422.8   
                       2027                    218231.0   
                       2028                    228028.9   
                  RAM  2025                    110843.9   
                       2026                    116453.3   
                       2027                    121598.5   
                       2028                    126694.6   
                  RAS  2025                    155088.7   
                       2026                    161941.7   
                       2027                    168539.0   
                       2028                    175201.8   
                  RAV  2025                    202569.4   
                       2026                    210888.4   
                       2027                    219810.6   
                       2028                    228770.4   
                  RSG  2025                    -91509.9   
                       2026                    -93647.4   
                       2027                    -95891.0   
                       2028                    -98421.6   
                  RTL  2025                    206431.2   
                       2026                    216238.3   
                       2027                    225803.3   
                       2028                    236048.6   
                  RTM  2025                    274071.5   
                       2026                    286505.5   
                       2027                    298381.9   
                       2028                    309959.6   
                  RTS  2025                    241880.6   
                       2026                    252222.4   
                       2027                    262432.8   
                       2028                    272461.6   
                  RTX  2025                    365354.7   
                       2026                    381336.9   
                       2027                    397489.7   
                       2028                    413026.5   
                  UCC  2025                   -299424.0   
                       2026                   -313151.1   
                       2027                   -326483.4   
                       2028                   -340066.3   
                  UDL  2025                  -1056293.1   
                       2026                  -1104736.8   
                       2027                  -1151759.5   
                       2028                  -1199674.0   
                  UDM  2025                   -476927.9   
                       2026                   -498798.0   
                       2027                   -520030.9   
                       2028                   -541663.0   
                  UDS  2025                   -222418.4   
                       2026                   -232622.8   
                       2027                   -242521.2   
                       2028                   -252608.7   
                  UDV  2025                  -1698195.1   
                       2026                  -1775773.4   
                       2027                  -1851318.3   
                       2028                  -1928434.1   
                  UFL  2025                   -904679.2   
                       2026                   -946165.5   
                       2027                   -986436.7   
                       2028                  -1027470.7   
                  UFM  2025                   -424214.3   
                       2026                   -444636.7   
                       2027                   -463586.4   
                       2028                   -483704.7   
                  UFS  2025                    -81437.2   
                       2026                    -86222.1   
                 

### Tasmania

In [11]:
display_data(sim_grants_20_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    150410.1   
                  2026                    154622.4   
                  2027                    158934.3   
                  2028                    164222.9   
             RAM  2025                     64120.4   
                  2026                     65697.1   
                  2027                     67534.5   
                  2028                     69306.8   
             RAS  2025                    128493.6   
                  2026                    132136.7   
                  2027                    135790.2   
                  2028                    139866.4   
             RAV  2025                     79462.0   
                  2026                     81691.9   
                  2027                     84300.9   
                  2028                     86575.0   
             UCC  2025                   -591722.8   
                  2026                   -608956.7   
                  2027                   -626868.2   
                  2028                   -647099.3   
             UFM  2025                   -546946.2   
                  2026                   -562844.4   
                  2027                   -579350.5   
                  2028                   -598003.7   
             UFS  2025                     44060.2   
                  2026                     45627.9   
                  2027                     46315.1   
                  2028                     47458.9   
             URM  2025                   -501049.9   
                  2026                   -514537.9   
                  2027                   -527564.9   
                  2028                   -542609.4   
             URS  2025                    115560.6   
                  2026                    118704.3   
                  2027                    122049.1   
                  2028                    126141.2   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   22.3     41480.0  
                  2026                                   22.9     41569.0  
                  2027                                   23.5     41646.0  
                  2028                                   24.2     41736.0  
             RAM  2025                                   13.6     23683.0  
                  2026                                   13.9     23735.0  
                  2027                                   14.3     23779.0  
                  2028                                   14.7     23831.0  
             RAS  2025                                   98.0      2672.0  
                  2026                                  100.6      2678.0  
                  2027                                  103.3      2683.0  
                  2028                                  106.2      2688.0  
             RAV  2025                                    6.0    105349.0  
                  2026                                    6.2    105628.0  
                  2027                                    6.3    105922.0  
                  2028                                    6.5    106243.0  
             UCC  2025                                   -8.7     68013.0  
                  2026                                   -8.9     68307.0  
                  2027                                   -9.1     68707.0  
                  2028                                   -9.4     69107.0  
             UFM  2025                                   -8.7    173305.0  
                  2026                                   -8.9    173977.0  
                  2027                                   -9.1    174856.0  
                  2028                                   -9.4    175746

### Northern Territory

In [12]:
display_data(sim_grants_20_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    253775.2   
                        2026                    262121.7   
                        2027                    270508.1   
                        2028                    280067.7   
                   RTM  2025                     18980.6   
                        2026                     19528.7   
                        2027                     20414.6   
                        2028                     21303.0   
                   RTS  2025                     -4393.9   
                        2026                     -4509.1   
                        2027                     -4614.4   
                        2028                     -4734.1   
                   RTX  2025                      2377.5   
                        2026                      2471.4   
                        2027                      2583.0   
                        2028                      2698.5   
                   UCC  2025                   -760500.0   
                        2026                   -787022.9   
                        2027                   -814704.0   
                        2028                   -846407.0   
                   UFS  2025                   -208357.5   
                        2026                   -215625.3   
                        2027                   -223209.7   
                        2028                   -231896.4   
                   URM  2025                   -365841.5   
                        2026                   -378594.0   
                        2027                   -391911.6   
                        2028                   -407160.7   
                   URS  2025                   -222907.0   
                        2026                   -228716.8   
                        2027                   -234002.1   
                        2028                   -239992.5   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   30.9   
                        2026                                   31.8   
                        2027                                   32.8   
                        2028                                   33.9   
                   RTM  2025                                    4.9   
                        2026                                    5.1   
                        2027                                    5.3   
                        2028                                    5.5   
                   RTS  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   RTX  2025                                   13.1   
                        2026                                   13.7   
                        2027                                   14.3   
                        2028                                   14.8   
                   UCC  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   UFS  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   URM  2025                                   -8.7   
                        2026                                   -8.9   
 

## A 10% Per Capita Minimum Grant

### New South Wales

In [13]:
display_data(sim_grants_10_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1119251.8   
                     2026                   1162505.5   
                     2027                   1206160.5   
                     2028                   1251113.2   
                RAM  2025                   1049926.2   
                     2026                   1088815.7   
                     2027                   1129918.3   
                     2028                   1168820.5   
                RAS  2025                   1449173.6   
                     2026                   1503607.3   
                     2027                   1558971.1   
                     2028                   1616999.3   
                RAV  2025                   1125803.8   
                     2026                   1167934.3   
                     2027                   1209798.6   
                     2028                   1256290.7   
                RTL  2025                   1658795.8   
                     2026                   1721845.5   
                     2027                   1792358.8   
                     2028                   1850286.8   
                RTM  2025                   1845095.1   
                     2026                   1912893.3   
                     2027                   1982855.6   
                     2028                   2061627.0   
                RTX  2025                     76851.1   
                     2026                     79806.2   
                     2027                     82759.9   
                     2028                     85215.2   
                UCC  2025                  -4165018.6   
                     2026                  -4329811.9   
                     2027                  -4497520.3   
                     2028                  -4669609.3   
                UDL  2025                  -1423822.9   
                     2026                  -1480162.1   
                     2027                  -1537492.3   
                     2028                  -1596323.1   
                UDM  2025                   -787566.6   
                     2026                   -818766.3   
                     2027                   -850513.4   
                     2028                   -883106.3   
                UDS  2025                   -228291.7   
                     2026                   -237389.4   
                     2027                   -246602.5   
                     2028                   -256100.4   
                UDV  2025                  -4435689.5   
                     2026                  -4611463.5   
                     2027                  -4790841.0   
                     2028                  -4974506.8   
                UFL  2025                  -1283750.0   
                     2026                  -1331196.5   
                     2027                  -1386355.5   
                     2028                  -1441204.8   
                UFM  2025                   -745242.2   
                     2026                   -774827.7   
                     2027                   -804976.4   
                     2028                   -836420.1   
                UFV  2025                  -2411049.9   
                     2026                  -2509184.3   
                     2027                  -2612439.5   
                     2028                  -2728564.1   
                URL  2025                    301384.6   
                     2026                    322633.3   
                     2027                    342244.0   
                     2028                    368063.3   
                URM  2025                    546519.7   
                     2026                    571378.9   
                     2027                    597184.7   
                     2028                    621693.8   
                URS  2025            

### Victoria

In [14]:
display_data(sim_grants_10_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    848291.5   
                  2026                    882246.9   
                  2027                    919235.5   
                  2028                    956076.9   
             RAM  2025                    951103.1   
                  2026                    989454.4   
                  2027                   1033622.7   
                  2028                   1076365.9   
             RAV  2025                    814152.1   
                  2026                    848545.7   
                  2027                    885637.5   
                  2028                    925126.0   
             UCC  2025                  -3245055.3   
                  2026                  -3387625.3   
                  2027                  -3531572.6   
                  2028                  -3678954.0   
             UDL  2025                  -1676903.8   
                  2026                  -1751554.5   
                  2027                  -1825729.8   
                  2028                  -1901733.2   
             UDV  2025                  -2009717.5   
                  2026                  -2103872.3   
                  2027                  -2196000.7   
                  2028                  -2289807.5   
             UFL  2025                    586109.7   
                  2026                    618707.1   
                  2027                    629937.9   
                  2028                    639384.7   
             UFM  2025                    -11569.6   
                  2026                     -9379.6   
                  2027                    -13655.8   
                  2028                    -14349.3   
             UFS  2025                    626121.7   
                  2026                    651537.4   
                  2027                    682096.7   
                  2028                    709958.6   
             UFV  2025                   -267868.0   
                  2026                   -288013.5   
                  2027                   -317726.5   
                  2028                   -334211.2   
             URL  2025                   1681567.9   
                  2026                   1777195.1   
                  2027                   1867372.1   
                  2028                   1944986.1   
             URM  2025                   1368983.5   
                  2026                   1430346.9   
                  2027                   1494825.2   
                  2028                   1557436.9   
             URS  2025                   1150728.7   
                  2026                   1201414.9   
                  2027                   1251806.6   
                  2028                   1306037.2   
             URV  2025                    606611.8   
                  2026                    681310.3   
                  2027                    730295.3   
                  2028                    763451.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                  120.7     50462.0  
                  2026                                  124.5     50851.0  
                  2027                                  128.9     51217.0  
                  2028                                  133.0     51602.0  
             RAM  2025                                  242.8      3917.0  
                  2026                                  250.7      3947.0  
                  2027                                  260.0      3976.0  
                  2028                                  268.8      4005.0  
             RAV  2025                                   52.6    246987.0  
                  2026                                 

### Queensland

In [15]:
display_data(sim_grants_10_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   2672025.8   
                  2026                   2781381.8   
                  2027                   2876717.3   
                  2028                   2992848.8   
             RAV  2025                   1666705.9   
                  2026                   1732214.3   
                  2027                   1804943.4   
                  2028                   1877399.9   
             RTL  2025                   2329912.0   
                  2026                   2421735.0   
                  2027                   2515252.0   
                  2028                   2609332.4   
             RTM  2025                   1704771.5   
                  2026                   1778079.2   
                  2027                   1848343.4   
                  2028                   1916833.3   
             RTS  2025                   1684804.2   
                  2026                   1752629.2   
                  2027                   1823284.5   
                  2028                   1897119.5   
             RTX  2025                   1670495.9   
                  2026                   1736580.7   
                  2027                   1804971.8   
                  2028                   1874020.3   
             UCC  2025                 -24039866.1   
                  2026                 -25073254.7   
                  2027                 -26094479.8   
                  2028                 -27138214.3   
             UFM  2025                   -705926.7   
                  2026                   -734832.8   
                  2027                   -764065.4   
                  2028                   -794154.9   
             UFS  2025                   -322155.0   
                  2026                   -336855.4   
                  2027                   -350768.2   
                  2028                   -365211.8   
             URL  2025                  -1977797.3   
                  2026                  -2054591.2   
                  2027                  -2131572.2   
                  2028                  -2209928.1   
             URM  2025                   -521229.6   
                  2026                   -540388.2   
                  2027                   -560330.1   
                  2028                   -579113.8   
             URS  2025                    512475.3   
                  2026                    538195.9   
                  2027                    557353.6   
                  2028                    577655.7   
             URV  2025                  -7299266.5   
                  2026                  -7594897.2   
                  2027                  -7889522.0   
                  2028                  -8189953.4   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  594.8      4492.0  
                  2026                                  611.6      4548.0  
                  2027                                  625.2      4601.0  
                  2028                                  643.1      4654.0  
             RAV  2025                                  125.1    105395.0  
                  2026                                  128.4    106725.0  
                  2027                                  132.2    107956.0  
                  2028                                  135.9    109216.0  
             RTL  2025                                  440.4     47507.0  
                  2026                                  452.1     48105.0  
                  2027                                  464.1     48660.0  
                  2028                                  475.8     49227.0  
             RTM  2025                     

### South Australia

In [16]:
display_data(sim_grants_10_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    425088.0   
                     2026                    440671.5   
                     2027                    455349.7   
                     2028                    470150.3   
                RAM  2025                    325166.1   
                     2026                    337200.1   
                     2027                    348734.9   
                     2028                    360985.8   
                RAS  2025                    268394.5   
                     2026                    277418.5   
                     2027                    286437.4   
                     2028                    295388.8   
                RAV  2025                    580767.1   
                     2026                    602362.3   
                     2027                    622660.9   
                     2028                    645303.7   
                RSG  2025                    -77859.6   
                     2026                    -79841.0   
                     2027                    -81760.8   
                     2028                    -83767.5   
                UCC  2025                   -501698.9   
                     2026                   -518895.8   
                     2027                   -536201.6   
                     2028                   -554145.0   
                UDL  2025                  -1949777.0   
                     2026                  -2016558.2   
                     2027                  -2083844.3   
                     2028                  -2153837.0   
                UDM  2025                   -900865.3   
                     2026                   -931747.9   
                     2027                   -962906.6   
                     2028                   -995185.8   
                UDS  2025                   -345425.7   
                     2026                   -357269.7   
                     2027                   -369191.7   
                     2028                   -381541.7   
                UDV  2025                  -1601845.7   
                     2026                  -1657766.9   
                     2027                  -1712049.1   
                     2028                  -1771118.6   
                UFL  2025                   1909022.5   
                     2026                   1951797.5   
                     2027                   2008469.9   
                     2028                   2072176.1   
                UFM  2025                   -752374.9   
                     2026                   -778156.9   
                     2027                   -804138.3   
                     2028                   -831030.6   
                UFS  2025                     49187.2   
                     2026                     51979.5   
                     2027                     57031.5   
                     2028                     61617.7   
                UFV  2025                  -1814370.8   
                     2026                  -1885621.2   
                     2027                  -1952498.5   
                     2028                  -2029331.8   
                URM  2025                   -469704.8   
                     2026                   -485588.4   
                     2027                   -501541.7   
                     2028                   -518302.9   
                URS  2025                    549970.5   
                     2026                    569828.1   
                     2027                    589541.2   
                     2028                    610832.4   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   50.1     83192.0  
                     2026  

### Western Australia

In [17]:
display_data(sim_grants_10_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    401360.6   
                       2026                    419639.8   
                       2027                    438579.8   
                       2028                    457354.1   
                  RAM  2025                    221105.3   
                       2026                    231344.3   
                       2027                    241653.4   
                       2028                    251897.1   
                  RAS  2025                    310094.0   
                       2026                    324344.4   
                       2027                    337549.7   
                       2028                    351294.0   
                  RAV  2025                    407068.9   
                       2026                    423416.2   
                       2027                    440921.8   
                       2028                    457370.5   
                  RSG  2025                   -182879.2   
                       2026                   -187074.2   
                       2027                   -191388.2   
                       2028                   -195206.6   
                  RTL  2025                    413492.5   
                       2026                    432863.5   
                       2027                    452988.2   
                       2028                    471658.3   
                  RTM  2025                    547553.7   
                       2026                    571663.6   
                       2027                    595101.8   
                       2028                    619186.7   
                  RTS  2025                    483528.7   
                       2026                    503756.0   
                       2027                    524848.2   
                       2028                    545582.3   
                  RTX  2025                    731694.7   
                       2026                    763913.1   
                       2027                    794793.6   
                       2028                    824221.3   
                  UCC  2025                   -598848.0   
                       2026                   -626302.1   
                       2027                   -652966.7   
                       2028                   -680132.7   
                  UDL  2025                  -2112586.2   
                       2026                  -2209473.7   
                       2027                  -2303518.9   
                       2028                  -2399348.0   
                  UDM  2025                   -953855.9   
                       2026                   -997595.9   
                       2027                  -1040061.8   
                       2028                  -1083325.9   
                  UDS  2025                   -444836.9   
                       2026                   -465245.6   
                       2027                   -485042.5   
                       2028                   -505217.5   
                  UDV  2025                  -3395662.3   
                       2026                  -3552322.8   
                       2027                  -3704066.5   
                       2028                  -3857696.6   
                  UFL  2025                  -1809358.5   
                       2026                  -1892331.1   
                       2027                  -1972873.5   
                       2028                  -2054941.4   
                  UFM  2025                   -847473.8   
                       2026                   -887813.1   
                       2027                   -927771.5   
                       2028                   -966492.7   
                  UFS  2025                   -163414.1   
                       2026                   -170554.4   
                 

### Tasmania

In [18]:
display_data(sim_grants_10_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    301207.8   
                  2026                    310449.5   
                  2027                    317584.9   
                  2028                    327220.7   
             RAM  2025                    128600.3   
                  2026                    132285.4   
                  2027                    135366.7   
                  2028                    140378.2   
             RAS  2025                    257175.9   
                  2026                    264430.6   
                  2027                    272084.7   
                  2028                    280155.6   
             RAV  2025                    159444.9   
                  2026                    162950.7   
                  2027                    168588.9   
                  2028                    173120.9   
             UCC  2025                  -1183445.7   
                  2026                  -1217913.3   
                  2027                  -1253736.4   
                  2028                  -1294198.5   
             UFM  2025                  -1093892.3   
                  2026                  -1125688.9   
                  2027                  -1158701.1   
                  2028                  -1196007.3   
             UFS  2025                     88150.8   
                  2026                     91503.9   
                  2027                     95497.0   
                  2028                     97765.5   
             URM  2025                  -1002099.9   
                  2026                  -1029075.9   
                  2027                  -1055129.7   
                  2028                  -1085218.8   
             URS  2025                    228523.5   
                  2026                    234253.4   
                  2027                    240623.2   
                  2028                    249071.8   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   44.7     41480.0  
                  2026                                   46.0     41569.0  
                  2027                                   47.0     41646.0  
                  2028                                   48.3     41736.0  
             RAM  2025                                   27.3     23683.0  
                  2026                                   28.1     23735.0  
                  2027                                   28.7     23779.0  
                  2028                                   29.6     23831.0  
             RAS  2025                                  196.2      2672.0  
                  2026                                  201.3      2678.0  
                  2027                                  206.8      2683.0  
                  2028                                  212.6      2688.0  
             RAV  2025                                   12.0    105349.0  
                  2026                                   12.3    105628.0  
                  2027                                   12.7    105922.0  
                  2028                                   13.0    106243.0  
             UCC  2025                                  -17.4     68013.0  
                  2026                                  -17.8     68307.0  
                  2027                                  -18.2     68707.0  
                  2028                                  -18.7     69107.0  
             UFM  2025                                  -17.4    173305.0  
                  2026                                  -17.8    173977.0  
                  2027                                  -18.2    174856.0  
                  2028                                  -18.7    175746

### Northern Territory

In [19]:
display_data(sim_grants_10_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    507247.6   
                        2026                    523810.5   
                        2027                    540694.8   
                        2028                    560016.2   
                   RTM  2025                     37991.0   
                        2026                     39308.6   
                        2027                     41130.9   
                        2028                     42808.1   
                   RTS  2025                     -8787.8   
                        2026                     -9018.2   
                        2027                     -9228.7   
                        2028                     -9468.2   
                   RTX  2025                      4781.6   
                        2026                      4985.1   
                        2027                      5127.3   
                        2028                      5303.6   
                   UCC  2025                  -1520999.9   
                        2026                  -1574045.8   
                        2027                  -1629408.0   
                        2028                  -1692814.0   
                   UFS  2025                   -416714.9   
                        2026                   -431250.6   
                        2027                   -446419.4   
                        2028                   -463792.8   
                   URM  2025                   -731683.0   
                        2026                   -757188.1   
                        2027                   -783823.2   
                        2028                   -814321.4   
                   URS  2025                   -445814.0   
                        2026                   -457378.3   
                        2027                   -468049.7   
                        2028                   -480039.8   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   61.7   
                        2026                                   63.6   
                        2027                                   65.5   
                        2028                                   67.8   
                   RTM  2025                                    9.8   
                        2026                                   10.2   
                        2027                                   10.7   
                        2028                                   11.2   
                   RTS  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   RTX  2025                                   26.4   
                        2026                                   27.5   
                        2027                                   28.3   
                        2028                                   29.1   
                   UCC  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   UFS  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   URM  2025                                  -17.4   
                        2026                                  -17.8   
 

## A World With No Per Capita Minimum Grant

### New South Wales

In [20]:
display_data(sim_grants_0_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1678644.0   
                     2026                   1742768.5   
                     2027                   1806188.9   
                     2028                   1872150.4   
                RAM  2025                   1572900.5   
                     2026                   1632548.9   
                     2027                   1692855.9   
                     2028                   1753792.3   
                RAS  2025                   2182511.4   
                     2026                   2257816.9   
                     2027                   2332784.4   
                     2028                   2411376.6   
                RAV  2025                   1692441.0   
                     2026                   1757071.7   
                     2027                   1821606.6   
                     2028                   1889700.4   
                RTL  2025                   2501841.4   
                     2026                   2594864.3   
                     2027                   2687315.6   
                     2028                   2790240.1   
                RTM  2025                   2776107.6   
                     2026                   2860614.7   
                     2027                   2955699.2   
                     2028                   3073849.3   
                RTX  2025                    116488.2   
                     2026                    121264.3   
                     2027                    125537.5   
                     2028                    129563.0   
                UCC  2025                  -6247527.8   
                     2026                  -6494717.9   
                     2027                  -6746280.4   
                     2028                  -7004414.0   
                UDL  2025                  -2135734.4   
                     2026                  -2220243.2   
                     2027                  -2306238.5   
                     2028                  -2394484.7   
                UDM  2025                  -1181434.1   
                     2026                  -1228304.3   
                     2027                  -1275954.4   
                     2028                  -1324798.2   
                UDS  2025                   -342372.8   
                     2026                   -355836.1   
                     2027                   -369941.9   
                     2028                   -384425.1   
                UDV  2025                  -6652781.0   
                     2026                  -6916793.8   
                     2027                  -7184748.6   
                     2028                  -7460153.9   
                UFL  2025                  -1925060.3   
                     2026                  -2007689.8   
                     2027                  -2091326.1   
                     2028                  -2169695.2   
                UFM  2025                  -1115018.2   
                     2026                  -1159674.1   
                     2027                  -1204900.5   
                     2028                  -1253546.0   
                UFV  2025                  -3629881.1   
                     2026                  -3784333.3   
                     2027                  -3930557.9   
                     2028                  -4092984.0   
                URL  2025                    454847.5   
                     2026                    486271.1   
                     2027                    520680.2   
                     2028                    552412.2   
                URM  2025                    820114.1   
                     2026                    857727.3   
                     2027                    897300.7   
                     2028                    933805.1   
                URS  2025            

### Victoria

In [21]:
display_data(sim_grants_0_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                   1270902.4   
                  2026                   1326625.2   
                  2027                   1381191.1   
                  2028                   1436028.0   
             RAM  2025                   1420600.5   
                  2026                   1478583.0   
                  2027                   1533071.6   
                  2028                   1596612.8   
             RAV  2025                   1220790.2   
                  2026                   1272530.2   
                  2027                   1326846.9   
                  2028                   1384435.1   
             UCC  2025                  -4867583.0   
                  2026                  -5081437.9   
                  2027                  -5297359.0   
                  2028                  -5518431.0   
             UDL  2025                  -2516861.8   
                  2026                  -2629245.4   
                  2027                  -2742520.0   
                  2028                  -2857626.0   
             UDV  2025                  -3015116.9   
                  2026                  -3151316.4   
                  2027                  -3280757.5   
                  2028                  -3424032.1   
             UFL  2025                    874877.2   
                  2026                    898415.0   
                  2027                    940669.6   
                  2028                    985925.9   
             UFM  2025                    -16715.3   
                  2026                    -20263.8   
                  2027                    -29244.6   
                  2028                    -29279.8   
             UFS  2025                    933208.1   
                  2026                    977056.9   
                  2027                   1018544.2   
                  2028                   1060355.2   
             UFV  2025                   -397127.1   
                  2026                   -423025.5   
                  2027                   -465309.8   
                  2028                   -506583.4   
             URL  2025                   2513220.8   
                  2026                   2640263.2   
                  2027                   2767892.7   
                  2028                   2894740.8   
             URM  2025                   2060219.2   
                  2026                   2153759.2   
                  2027                   2249395.5   
                  2028                   2344485.5   
             URS  2025                   1728584.0   
                  2026                   1804921.0   
                  2027                   1878603.2   
                  2028                   1962674.1   
             URV  2025                    886969.8   
                  2026                    981762.8   
                  2027                   1075604.8   
                  2028                   1207678.1   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                  180.9     50462.0  
                  2026                                  187.4     50851.0  
                  2027                                  193.8     51217.0  
                  2028                                  199.9     51602.0  
             RAM  2025                                  362.7      3917.0  
                  2026                                  374.6      3947.0  
                  2027                                  385.6      3976.0  
                  2028                                  398.7      4005.0  
             RAV  2025                                   79.0    246987.0  
                  2026                                 

### Queensland

In [22]:
display_data(sim_grants_0_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   4062653.5   
                  2026                   4229440.4   
                  2027                   4400465.6   
                  2028                   4557257.0   
             RAV  2025                   2496063.6   
                  2026                   2598877.0   
                  2027                   2700091.2   
                  2028                   2797658.5   
             RTL  2025                   3488180.0   
                  2026                   3637023.9   
                  2027                   3780571.4   
                  2028                   3927301.1   
             RTM  2025                   2562064.1   
                  2026                   2669176.4   
                  2027                   2776766.8   
                  2028                   2887653.8   
             RTS  2025                   2533760.7   
                  2026                   2635015.9   
                  2027                   2736292.8   
                  2028                   2838587.6   
             RTX  2025                   2499624.2   
                  2026                   2600678.5   
                  2027                   2700063.3   
                  2028                   2803384.5   
             UCC  2025                 -36059799.2   
                  2026                 -37609882.1   
                  2027                 -39141719.8   
                  2028                 -40707321.6   
             UFM  2025                  -1059168.1   
                  2026                  -1103736.1   
                  2027                  -1147205.9   
                  2028                  -1191781.9   
             UFS  2025                   -482358.5   
                  2026                   -502572.8   
                  2027                   -522910.8   
                  2028                   -543627.4   
             URL  2025                  -2966695.9   
                  2026                  -3081886.9   
                  2027                  -3197358.3   
                  2028                  -3314892.2   
             URM  2025                   -782786.5   
                  2026                   -813256.8   
                  2027                   -841945.0   
                  2028                   -871367.7   
             URS  2025                    763554.2   
                  2026                    790387.0   
                  2027                    821533.2   
                  2028                    857396.2   
             URV  2025                 -10948899.8   
                  2026                 -11392345.8   
                  2027                 -11834283.0   
                  2028                 -12284930.1   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  904.4      4492.0  
                  2026                                  930.0      4548.0  
                  2027                                  956.4      4601.0  
                  2028                                  979.2      4654.0  
             RAV  2025                                  187.4    105395.0  
                  2026                                  192.6    106725.0  
                  2027                                  197.8    107956.0  
                  2028                                  202.6    109216.0  
             RTL  2025                                  660.1     47507.0  
                  2026                                  679.3     48105.0  
                  2027                                  698.0     48660.0  
                  2028                                  716.7     49227.0  
             RTM  2025                     

### South Australia

In [23]:
display_data(sim_grants_0_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    637055.6   
                     2026                    658409.9   
                     2027                    680382.5   
                     2028                    703028.2   
                RAM  2025                    488438.5   
                     2026                    504134.4   
                     2027                    521120.0   
                     2028                    538475.8   
                RAS  2025                    402787.6   
                     2026                    416637.3   
                     2027                    429692.2   
                     2028                    443696.3   
                RAV  2025                    873618.7   
                     2026                    907025.2   
                     2027                    936762.3   
                     2028                    968165.8   
                RSG  2025                   -116551.7   
                     2026                   -119496.5   
                     2027                   -122732.6   
                     2028                   -125709.1   
                UCC  2025                   -752548.4   
                     2026                   -778343.7   
                     2027                   -804302.4   
                     2028                   -831217.4   
                UDL  2025                  -2924476.2   
                     2026                  -3024815.8   
                     2027                  -3125571.9   
                     2028                  -3229609.2   
                UDM  2025                  -1351333.0   
                     2026                  -1397684.0   
                     2027                  -1444452.7   
                     2028                  -1492816.4   
                UDS  2025                   -518138.6   
                     2026                   -535904.6   
                     2027                   -553787.6   
                     2028                   -572312.5   
                UDV  2025                  -2395047.5   
                     2026                  -2482611.9   
                     2027                  -2571306.8   
                     2028                  -2662910.0   
                UFL  2025                   2842146.4   
                     2026                   2930421.2   
                     2027                   3025307.9   
                     2028                   3123272.1   
                UFM  2025                  -1128562.3   
                     2026                  -1167235.4   
                     2027                  -1206207.5   
                     2028                  -1246545.9   
                UFS  2025                     71277.4   
                     2026                     78291.8   
                     2027                     86124.3   
                     2028                     94193.8   
                UFV  2025                  -2715899.6   
                     2026                  -2811254.4   
                     2027                  -2909561.7   
                     2028                  -3009048.0   
                URM  2025                   -708872.1   
                     2026                   -736119.9   
                     2027                   -761798.0   
                     2028                   -786560.1   
                URS  2025                    824396.7   
                     2026                    854277.8   
                     2027                    886499.4   
                     2028                    918322.7   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   75.1     83192.0  
                     2026  

### Western Australia

In [24]:
display_data(sim_grants_0_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    600891.3   
                       2026                    628236.9   
                       2027                    655206.1   
                       2028                    684390.7   
                  RAM  2025                    331961.6   
                       2026                    348192.7   
                       2027                    364025.3   
                       2028                    380139.8   
                  RAS  2025                    464741.7   
                       2026                    485743.4   
                       2027                    505662.9   
                       2028                    525983.4   
                  RAV  2025                    612023.2   
                       2026                    634557.4   
                       2027                    656561.7   
                       2028                    682739.8   
                  RSG  2025                   -274719.8   
                       2026                   -282175.5   
                       2027                   -288851.7   
                       2028                   -295032.4   
                  RTL  2025                    619852.8   
                       2026                    650617.1   
                       2027                    678553.3   
                       2028                    709656.5   
                  RTM  2025                    821557.4   
                       2026                    857643.7   
                       2027                    890249.4   
                       2028                    925565.2   
                  RTS  2025                    723193.2   
                       2026                    755216.8   
                       2027                    787190.5   
                       2028                    817312.9   
                  RTX  2025                   1099721.7   
                       2026                   1149454.3   
                       2027                   1198123.0   
                       2028                   1244077.6   
                  UCC  2025                   -898272.0   
                       2026                   -939453.2   
                       2027                   -979450.1   
                       2028                  -1020199.0   
                  UDL  2025                  -3168879.3   
                       2026                  -3314210.5   
                       2027                  -3455278.4   
                       2028                  -3599022.0   
                  UDM  2025                  -1430783.8   
                       2026                  -1496393.9   
                       2027                  -1560092.7   
                       2028                  -1624988.9   
                  UDS  2025                   -667255.3   
                       2026                   -697868.5   
                       2027                   -727563.7   
                       2028                   -757826.2   
                  UDV  2025                  -5094461.5   
                       2026                  -5329928.2   
                       2027                  -5557249.7   
                       2028                  -5791058.1   
                  UFL  2025                  -2714037.7   
                       2026                  -2838496.6   
                       2027                  -2959310.2   
                       2028                  -3082412.1   
                  UFM  2025                  -1273712.6   
                       2026                  -1332329.5   
                       2027                  -1392892.7   
                       2028                  -1451386.9   
                  UFS  2025                   -246533.1   
                       2026                   -262627.5   
                 

### Tasmania

In [25]:
display_data(sim_grants_0_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    452174.1   
                  2026                    464704.4   
                  2027                    477617.2   
                  2028                    493694.3   
             RAM  2025                    191650.3   
                  2026                    197796.2   
                  2027                    203424.4   
                  2028                    211236.8   
             RAS  2025                    385199.8   
                  2026                    395791.0   
                  2027                    409720.4   
                  2028                    421591.7   
             RAV  2025                    239844.2   
                  2026                    246673.1   
                  2027                    251180.1   
                  2028                    258539.3   
             UCC  2025                  -1775168.5   
                  2026                  -1826870.0   
                  2027                  -1880604.5   
                  2028                  -1941297.8   
             UFM  2025                  -1640838.5   
                  2026                  -1688533.3   
                  2027                  -1738051.6   
                  2028                  -1794011.0   
             UFS  2025                    132701.6   
                  2026                    136575.9   
                  2027                    140048.8   
                  2028                    143552.7   
             URM  2025                  -1503149.8   
                  2026                  -1543613.8   
                  2027                  -1582694.6   
                  2028                  -1627828.2   
             URS  2025                    342079.0   
                  2026                    350746.4   
                  2027                    362988.3   
                  2028                    371826.8   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   67.2     41480.0  
                  2026                                   68.9     41569.0  
                  2027                                   70.7     41646.0  
                  2028                                   72.9     41736.0  
             RAM  2025                                   40.8     23683.0  
                  2026                                   41.9     23735.0  
                  2027                                   43.0     23779.0  
                  2028                                   44.5     23831.0  
             RAS  2025                                  294.1      2672.0  
                  2026                                  301.5      2678.0  
                  2027                                  311.6      2683.0  
                  2028                                  320.1      2688.0  
             RAV  2025                                   18.1    105349.0  
                  2026                                   18.6    105628.0  
                  2027                                   19.0    105922.0  
                  2028                                   19.5    106243.0  
             UCC  2025                                  -26.1     68013.0  
                  2026                                  -26.7     68307.0  
                  2027                                  -27.4     68707.0  
                  2028                                  -28.1     69107.0  
             UFM  2025                                  -26.1    173305.0  
                  2026                                  -26.7    173977.0  
                  2027                                  -27.4    174856.0  
                  2028                                  -28.1    175746

### Northern Territory

In [26]:
display_data(sim_grants_0_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    761006.2   
                        2026                    785692.2   
                        2027                    810990.7   
                        2028                    839706.1   
                   RTM  2025                     56604.9   
                        2026                     58758.7   
                        2027                     60815.5   
                        2028                     63100.7   
                   RTS  2025                    -13181.7   
                        2026                    -13527.3   
                        2027                    -13843.1   
                        2028                    -14202.3   
                   RTX  2025                      7167.7   
                        2026                      7383.0   
                        2027                      7626.7   
                        2028                      7944.5   
                   UCC  2025                  -2281499.9   
                        2026                  -2361068.7   
                        2027                  -2444112.0   
                        2028                  -2539221.0   
                   UFS  2025                   -625072.4   
                        2026                   -646875.8   
                        2027                   -669629.1   
                        2028                   -695689.3   
                   URM  2025                  -1097524.5   
                        2026                  -1135782.1   
                        2027                  -1175734.9   
                        2028                  -1221482.0   
                   URS  2025                   -668748.5   
                        2026                   -686025.8   
                        2027                   -701743.5   
                        2028                   -719694.2   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   92.6   
                        2026                                   95.4   
                        2027                                   98.3   
                        2028                                  101.7   
                   RTM  2025                                   14.6   
                        2026                                   15.2   
                        2027                                   15.8   
                        2028                                   16.4   
                   RTS  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   RTX  2025                                   39.6   
                        2026                                   40.8   
                        2027                                   42.1   
                        2028                                   43.7   
                   UCC  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   UFS  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   URM  2025                                  -26.1   
                        2026                                  -26.7   
 

## And for sanity reasons, status quo

### New South Wales

In [27]:
display_data(sim_grants_30_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTX  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025            

### Victoria

In [28]:
display_data(sim_grants_30_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                    0.0     50462.0  
                  2026                                    0.0     50851.0  
                  2027                                    0.0     51217.0  
                  2028                                    0.0     51602.0  
             RAM  2025                                    0.0      3917.0  
                  2026                                    0.0      3947.0  
                  2027                                    0.0      3976.0  
                  2028                                    0.0      4005.0  
             RAV  2025                                    0.0    246987.0  
                  2026                                 

### Queensland

In [29]:
display_data(sim_grants_30_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTX  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                    0.0      4492.0  
                  2026                                    0.0      4548.0  
                  2027                                    0.0      4601.0  
                  2028                                    0.0      4654.0  
             RAV  2025                                    0.0    105395.0  
                  2026                                    0.0    106725.0  
                  2027                                    0.0    107956.0  
                  2028                                    0.0    109216.0  
             RTL  2025                                    0.0     47507.0  
                  2026                                    0.0     48105.0  
                  2027                                    0.0     48660.0  
                  2028                                    0.0     49227.0  
             RTM  2025                     

### South Australia

In [30]:
display_data(sim_grants_30_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RSG  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                    0.0     83192.0  
                     2026  

### Western Australia

In [31]:
display_data(sim_grants_30_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RSG  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTX  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UCC  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFS  2025                         0.0   
                       2026                         0.0   
                 

### Tasmania

In [32]:
display_data(sim_grants_30_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                    0.0     41480.0  
                  2026                                    0.0     41569.0  
                  2027                                    0.0     41646.0  
                  2028                                    0.0     41736.0  
             RAM  2025                                    0.0     23683.0  
                  2026                                    0.0     23735.0  
                  2027                                    0.0     23779.0  
                  2028                                    0.0     23831.0  
             RAS  2025                                    0.0      2672.0  
                  2026                                    0.0      2678.0  
                  2027                                    0.0      2683.0  
                  2028                                    0.0      2688.0  
             RAV  2025                                    0.0    105349.0  
                  2026                                    0.0    105628.0  
                  2027                                    0.0    105922.0  
                  2028                                    0.0    106243.0  
             UCC  2025                                    0.0     68013.0  
                  2026                                    0.0     68307.0  
                  2027                                    0.0     68707.0  
                  2028                                    0.0     69107.0  
             UFM  2025                                    0.0    173305.0  
                  2026                                    0.0    173977.0  
                  2027                                    0.0    174856.0  
                  2028                                    0.0    175746

### Northern Territory

In [33]:
display_data(sim_grants_30_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTX  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UCC  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UFS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTM  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTS  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTX  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   UCC  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   UFS  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   URM  2025                                    0.0   
                        2026                                    0.0   
 